In [ ]:
!pip install torch torchaudio

In [ ]:
!pip install pydub

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
from torchaudio import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import librosa
from torchaudio.transforms import MelSpectrogram
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/'Colab Notebooks'/'NPB198_AudioClassification_Project'

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/NPB198_AudioClassification_Project'
/content/drive/MyDrive/Colab Notebooks/NPB198_AudioClassification_Project


In [ ]:
os.listdir()
#dataset_commoonvoice = datasets.COMMONVOICE('./audiodata_commonvoice', tsv="train.tsv", download=True,  )

['NPB198_AudioClassification.ipynb',
 'archive_age_gender_precleaned_data',
 'archive_speaker_data',
 'NPB198_AudioClassification_Group.ipynb',
 'audio_mel_spect_list.pt',
 'NPB198_AudioClassification_Presentation.gslides']

In [ ]:
speakers_info = pd.read_csv("./archive_speaker_data/speakers_all.csv")
# if file_missing? column is false, then exists video, filename column is correct
print(speakers_info.head())

    age  age_onset              birthplace  filename native_language   sex  \
0  24.0       12.0         koussi, senegal   balanta         balanta  male   
1  18.0       10.0          buea, cameroon  cameroon        cameroon  male   
2  48.0        8.0  hong, adamawa, nigeria  fulfulde        fulfulde  male   
3  42.0       42.0   port-au-prince, haiti   haitian         haitian  male   
4  40.0       35.0   port-au-prince, haiti   haitian         haitian  male   

   speakerid   country  file_missing?  Unnamed: 9  Unnamed: 10 Unnamed: 11  
0        788   senegal           True         NaN          NaN         NaN  
1       1953  cameroon           True         NaN          NaN         NaN  
2       1037   nigeria           True         NaN          NaN         NaN  
3       1165     haiti           True         NaN          NaN         NaN  
4       1166     haiti           True         NaN          NaN         NaN  


In [ ]:
# convert .mp3 files to .wav files
from pydub import AudioSegment
audio_files_to_convert = os.listdir("archive_speaker_data/recordings/recordings/")
total = len(audio_files_to_convert)
i = 1
for one_audio_file in audio_files_to_convert:
  print( str(i) + " / " + str(total))
  i = i + 1
  dst_audio_file = "archive_speaker_data/recordings/recordings_wav/" +  one_audio_file[ 0 : one_audio_file.index(".") ] + ".wav"
  audio_segment = AudioSegment.from_mp3( "archive_speaker_data/recordings/recordings/" + one_audio_file)
  audio_segment.export(dst_audio_file, format = 'wav')



In [ ]:
speakers_valid_df = speakers_info.loc[ speakers_info['file_missing?'] == False ]
speakers_valid_df = speakers_valid_df.loc[ speakers_valid_df['filename'] != "nicaragua" ] #-> some invalid data exists still

In [ ]:
def open_audio_file(audio_file_name):
  file_to_load_loc = "archive_speaker_data/recordings/recordings_wav/" +  audio_file_name + ".wav"
  try:
    signal, sample_rate = torchaudio.load( file_to_load_loc )
  except:
    return( None )
  return (signal, sample_rate)

open_audio_file_vec = np.vectorize(open_audio_file)

signal_sample_rate_info = speakers_valid_df[ 'filename' ].map( open_audio_file_vec )


In [ ]:
# first attempt
first_audio_signal = np.array( np.array(signal_sample_rate_info)[0][0][0] )
first_audio_sample_rate = np.array(signal_sample_rate_info)[0][1]
print( first_audio_signal )
print( first_audio_sample_rate )


spec = transforms.MelSpectrogram( first_audio_sample_rate , n_fft=1024, hop_length= None, n_mels= 64)
spec = transforms.AmplitudeToDB(top_db=80)

# second attempt

n_fft = 2048
win_len = None
hop_len = 512
n_mels = 128
sample_rate = 6000

path = 'archive_speaker_data/recordings/recordings_wav/tamil1.wav'

waveform, sample_rate = librosa.load(path, sr=sample_rate)
waveform = torch.Tensor(waveform)

torchaudio_melspec = MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=n_fft,
    win_length=win_len,
    hop_length=hop_len,
    center=True,
    pad_mode="reflect",
    power=2.0,
    norm='slaney',
    onesided=True,
    n_mels=n_mels,
)(waveform)

fig, axs = plt.subplots(1, figsize=(20, 5))
fig.suptitle('Mel Spectrogram')

axs.set_title('torchaudio')
axs.set_ylabel('mel bin')
axs.set_xlabel('frame')
axs.imshow(librosa.power_to_db( torchaudio_melspec ), aspect='auto')
plt.show()

print( torchaudio_melspec )

In [ ]:
# organize data to train
print( speakers_valid_df.shape  )
print( speakers_valid_df.columns )
#print(  signal_sample_rate_info.shape )

(2139, 12)
Index(['age', 'age_onset', 'birthplace', 'filename', 'native_language', 'sex',
       'speakerid', 'country', 'file_missing?', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11'],
      dtype='object')


In [ ]:
print( type(signal_sample_rate_info) )
print( type(np.array( signal_sample_rate_info )[0]) )
print(signal_sample_rate_info.count)

In [ ]:
print( signal_sample_rate_info.iloc[0] )

(array([[ 0.0090332 ,  0.01101685,  0.00881958, ..., -0.00131226,
        -0.00134277, -0.00088501]], dtype=float32), array(44100))


In [ ]:
print( type( signal_sample_rate_info.iloc[0] ) )

<class 'tuple'>


In [ ]:
print( (signal_sample_rate_info.iloc[0])[0] )
print( (signal_sample_rate_info.iloc[0])[1] )
# tuple two - signal and sample rate

[[ 0.0090332   0.01101685  0.00881958 ... -0.00131226 -0.00134277
  -0.00088501]]
44100


In [ ]:
def generate_audio_mel_spectograms( file_name ):
  n_fft = 2048
  win_len = None
  hop_len = 512
  n_mels = 128
  sample_rate = 6000

  try :
      path = 'archive_speaker_data/recordings/recordings_wav/' + file_name + '.wav'

      waveform, sample_rate = librosa.load(path, sr=sample_rate)
      waveform = torch.Tensor(waveform)

      torchaudio_melspec = MelSpectrogram(
          sample_rate=sample_rate,
          n_fft=n_fft,
          win_length=win_len,
          hop_length=hop_len,
          center=True,
          pad_mode="reflect",
          power=2.0,
          norm='slaney',
          onesided=True,
          n_mels=n_mels,
      )(waveform)

      return( torchaudio_melspec )
  except :
      print(file_name)
      return None

  

generate_audio_mel_spectograms_vec = np.vectorize( generate_audio_mel_spectograms )
audio_mel_spectograms_list = speakers_valid_df['filename'].map( generate_audio_mel_spectograms_vec )

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


sinhalese1
sinhalese1


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [ ]:
print( type(audio_mel_spectograms_list) )
print( audio_mel_spectograms_list.iloc[0] )

<class 'pandas.core.series.Series'>
[[2.3143284e-02 2.3346584e-02 2.0850325e-02 ... 3.3120535e-02
  6.5729646e-03 3.6904488e-03]
 [5.4113071e-02 5.5285510e-02 2.6749350e-02 ... 1.7494136e-02
  7.9168929e-03 5.0841928e-03]
 [5.2709367e-02 6.5688714e-02 6.3287415e-02 ... 4.9043693e-02
  1.9238539e-02 3.5183731e-02]
 ...
 [1.7940642e-05 5.1856264e-06 5.8324156e-07 ... 9.5549194e-07
  1.5065723e-06 3.4009627e-06]
 [2.0122719e-05 5.1563179e-06 7.3431046e-08 ... 1.3015388e-07
  1.8934415e-07 1.6570639e-06]
 [1.9295801e-05 4.8292595e-06 3.2188896e-09 ... 3.7291485e-09
  7.3043957e-08 1.4874594e-06]]


In [ ]:
torch.save( audio_mel_spectograms_list, "audio_mel_spect_list.pt" )

In [ ]:
loaded_audio_mel_spect_list = torch.load( "audio_mel_spect_list.pt" )

In [ ]:
print(loaded_audio_mel_spect_list.iloc[0])

[[2.3143284e-02 2.3346584e-02 2.0850325e-02 ... 3.3120535e-02
  6.5729646e-03 3.6904488e-03]
 [5.4113071e-02 5.5285510e-02 2.6749350e-02 ... 1.7494136e-02
  7.9168929e-03 5.0841928e-03]
 [5.2709367e-02 6.5688714e-02 6.3287415e-02 ... 4.9043693e-02
  1.9238539e-02 3.5183731e-02]
 ...
 [1.7940642e-05 5.1856264e-06 5.8324156e-07 ... 9.5549194e-07
  1.5065723e-06 3.4009627e-06]
 [2.0122719e-05 5.1563179e-06 7.3431046e-08 ... 1.3015388e-07
  1.8934415e-07 1.6570639e-06]
 [1.9295801e-05 4.8292595e-06 3.2188896e-09 ... 3.7291485e-09
  7.3043957e-08 1.4874594e-06]]


In [ ]:
print( type( loaded_audio_mel_spect_list ) )
print( loaded_audio_mel_spect_list.size )
print( len(speakers_valid_df) )

<class 'pandas.core.series.Series'>
2139
2139


In [ ]:
audio_mel_subset = loaded_audio_mel_spect_list.loc[0:500]
speakers_valid_df_subset = speakers_valid_df.loc[0:500,:]
print( audio_mel_subset.size )
print( len( speakers_valid_df_subset ) )
print( type( audio_mel_subset.iloc[0] ) )
print( len( audio_mel_subset.iloc[0] ) )
speakers_valid_df_subset['melspectogram'] = audio_mel_subset
print( speakers_valid_df_subset.iloc[0] )
speakers_valid_df_subset = speakers_valid_df_subset[[ "melspectogram",  "native_language" ]]
print( "\n smaller df: \n" )
print( speakers_valid_df_subset.iloc[0] )

469
469
<class 'numpy.ndarray'>
128
age                                                             27.0
age_onset                                                        9.0
birthplace                                    virginia, south africa
filename                                                  afrikaans1
native_language                                            afrikaans
sex                                                           female
speakerid                                                          1
country                                                 south africa
file_missing?                                                  False
Unnamed: 9                                                       NaN
Unnamed: 10                                                      NaN
Unnamed: 11                                                      NaN
melspectogram      [[0.023143284, 0.023346584, 0.020850325, 0.013...
Name: 32, dtype: object

 smaller df: 

melspectogram      [[0.0231

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
print(speakers_valid_df_subset.head())
print( speakers_valid_df_subset.index )

                                        melspectogram native_language
32  [[0.023143284, 0.023346584, 0.020850325, 0.013...       afrikaans
33  [[5.584354e-06, 5.6655367e-06, 5.7940765e-06, ...       afrikaans
34  [[0.0004151349, 0.00029746164, 8.756204e-05, 6...       afrikaans
35  [[0.0011264905, 0.001081302, 0.0011223338, 0.0...       afrikaans
36  [[0.111883625, 0.40863723, 0.76552546, 1.55740...       afrikaans
Int64Index([ 32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
            ...
            491, 492, 493, 494, 495, 496, 497, 498, 499, 500],
           dtype='int64', length=469)


In [ ]:
from matplotlib import test
# training set
# test set: about 50
full_indices = speakers_valid_df_subset.index
indices_test = np.empty(0)
actual_indices_labels_test = np.empty(0)
seen_languages = np.empty(0)
added_languages = np.empty(0)
for i in range(0, len(speakers_valid_df_subset)):
  cur_language = speakers_valid_df_subset.iloc[i, 1]
  if cur_language in seen_languages and cur_language not in added_languages:
    actual_indices_labels_test = np.append( actual_indices_labels_test, full_indices[ i ] )
    indices_test = np.append( indices_test, i )
    added_languages = np.append( added_languages, cur_language )
  if cur_language not in seen_languages:
    seen_languages = np.append( seen_languages, cur_language )

test_speakers_df = speakers_valid_df_subset.iloc[ indices_test ,:]
train_speakers_df = speakers_valid_df_subset.drop( labels = actual_indices_labels_test , axis = 0, inplace = False )

print( "test dataset: " )
print( test_speakers_df.head() )
print( "training dataset: " )
print( train_speakers_df.head() )
print( "len test dataset: " )
print( len( test_speakers_df ) )
print( "len training dataset: " )
print( len(train_speakers_df) )


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  from ipykernel import kernelapp as app


test dataset: 
                                        melspectogram native_language
33  [[5.584354e-06, 5.6655367e-06, 5.7940765e-06, ...       afrikaans
40  [[2.7116478e-06, 2.6871908e-06, 2.6501168e-06,...        albanian
49  [[0.6141132, 0.69792235, 0.60047567, 0.4714463...         amazigh
51  [[5.2559983e-05, 0.0001005459, 7.495135e-05, 6...         amharic
71  [[0.38766858, 0.3925465, 0.39441636, 0.3912688...          arabic
training dataset: 
                                        melspectogram native_language
32  [[0.023143284, 0.023346584, 0.020850325, 0.013...       afrikaans
34  [[0.0004151349, 0.00029746164, 8.756204e-05, 6...       afrikaans
35  [[0.0011264905, 0.001081302, 0.0011223338, 0.0...       afrikaans
36  [[0.111883625, 0.40863723, 0.76552546, 1.55740...       afrikaans
37  [[0.044271175, 0.0352975, 0.034747146, 0.03011...            agni
len test dataset: 
26
len training dataset: 
443


In [ ]:
print( seen_languages )
print( len(seen_languages) )

['afrikaans' 'agni' 'akan' 'albanian' 'amazigh' 'amharic' 'arabic'
 'armenian' 'ashanti' 'azerbaijani' 'bafang' 'baga' 'bai' 'bambara'
 'bamun' 'bari' 'basque' 'bavarian' 'belarusan' 'bengali' 'bosnian'
 'bulgarian' 'burmese' 'cantonese' 'carolinian' 'catalan' 'cebuano'
 'chaldean' 'chamorro' 'chichewa' 'chittagonian' 'croatian' 'czech'
 'danish' 'dari' 'dinka' 'dutch' 'ebira' 'edo' 'english']
40


In [ ]:
map_str_indices = range(0, len(seen_languages))
native_language_codes_train = np.empty(0)
for i in range(0, len(train_speakers_df)):
  native_language_codes_train = np.append( native_language_codes_train, np.where( seen_languages == train_speakers_df.iloc[i, 1] )[0][0] )

native_language_codes_test = np.empty(0)
for i in range(0, len(test_speakers_df)):
  native_language_codes_test = np.append( native_language_codes_test, np.where( seen_languages == test_speakers_df.iloc[i, 1] )[0][0] )



In [ ]:
# need to shorten length of each audio file
def len_arr( audio_data_arr ):
  return len( audio_data_arr[0] )
len_arr_vec = np.vectorize( len_arr )

train_speakers_audio_data_len_arr = len_arr_vec( np.array(train_speakers_df["melspectogram"]) )
train_audio_data_min = min( train_speakers_audio_data_len_arr )
print( train_speakers_audio_data_len_arr )
print( train_audio_data_min )
print()

test_speakers_audio_data_len_arr = len_arr_vec( np.array(test_speakers_df["melspectogram"]) )
test_audio_data_min = min( test_speakers_audio_data_len_arr )
print( test_speakers_audio_data_len_arr )
print( test_audio_data_min )

total_min = min( test_audio_data_min, train_audio_data_min )
print()
print(total_min)

[244 316 276 238 412 296 348 243 354 256 287 301 318 280 501 448 327 308
 383 353 392 341 296 276 276 623 455 281 513 306 291 377 553 491 533 352
 443 229 382 465 355 574 438 238 264 333 449 329 312 325 334 334 482 382
 468 775 998 418 565 275 575 325 477 264 353 452 243 375 383 273 412 434
 760 367 380 483 429 753 411 460 369 463 303 305 374 279 390 311 292 314
 299 309 459 254 573 504 294 351 556 465 429 337 355 352 377 321 365 350
 250 307 280 243 279 334 449 339 380 361 332 300 246 408 341 333 407 443
 656 520 586 412 546 369 288 265 513 284 403 338 314 185 410 335 274 343
 305 478 494 713 418 403 384 364 374 297 756 280 296 305 338 292 371 318
 272 569 368 267 227 324 353 412 241 430 350 373 391 252 286 334 336 372
 342 236 306 389 282 334 428 320 271 318 262 471 285 335 335 225 326 410
 288 388 400 275 286 330 240 270 253 292 269 402 496 398 308 277 319 260
 413 404 353 313 226 328 368 327 271 218 240 285 242 426 621 399 277 303
 289 335 255 349 284 261 288 401 273 311 334 331 33

In [ ]:
# make mono channel
# trim

num_arrs_in_one_audio_file_train = len( train_speakers_df.iloc[0, 0] )
mel_audio_trimmed_train = np.array( [ [ [[0] * total_min] * num_arrs_in_one_audio_file_train ] ] )

#print(mel_audio_trimmed_train)
#print()

for i in range(0, len(train_speakers_df)):
  cur_audio = train_speakers_df.iloc[0, 0]
  new_audio_arr = np.array( [ cur_audio[0][0:total_min ] ] )
  for j in range(1, len(cur_audio)):
    new_audio_arr = np.vstack( [ new_audio_arr, cur_audio[ j ][ 0:total_min ] ] )
  #print(new_audio_arr)
  #print(len(new_audio_arr))
  mel_audio_trimmed_train = np.vstack( (mel_audio_trimmed_train, [ [new_audio_arr] ] ) )
mel_audio_trimmed_train = mel_audio_trimmed_train[1:]
print(mel_audio_trimmed_train[0])



num_arrs_in_one_audio_file_test = len( test_speakers_df.iloc[0, 0] )
mel_audio_trimmed_test  = np.array( [ [ [[0] * total_min] * num_arrs_in_one_audio_file_test  ] ] )

#print(mel_audio_trimmed_test)
#print()

for i in range(0, len(test_speakers_df)):
  cur_audio = test_speakers_df.iloc[0, 0]
  new_audio_arr = np.array( [ cur_audio[0][0:total_min ] ] )
  for j in range(1, len(cur_audio)):
    new_audio_arr = np.vstack( [ new_audio_arr, cur_audio[ j ][ 0:total_min ] ] )
  #print(new_audio_arr)
  #print(len(new_audio_arr))
  mel_audio_trimmed_test = np.vstack( (mel_audio_trimmed_test, [ [new_audio_arr] ] ) )
mel_audio_trimmed_test = mel_audio_trimmed_test[1:]
print(mel_audio_trimmed_test[0])


[[[2.31432840e-02 2.33465843e-02 2.08503250e-02 ... 3.09503302e-02
   1.68734062e-02 1.82807483e-02]
  [5.41130714e-02 5.52855097e-02 2.67493501e-02 ... 6.82429895e-02
   1.94838773e-02 1.41601451e-02]
  [5.27093671e-02 6.56887144e-02 6.32874146e-02 ... 1.37286052e-01
   3.48061286e-02 2.68036909e-02]
  ...
  [1.79406416e-05 5.18562638e-06 5.83241558e-07 ... 2.52669008e-04
   3.99608514e-04 4.04445746e-04]
  [2.01227194e-05 5.15631791e-06 7.34310461e-08 ... 6.45049222e-05
   4.30963955e-05 4.72981264e-05]
  [1.92958014e-05 4.82925952e-06 3.21888960e-09 ... 3.05295066e-06
   1.78964683e-06 6.75350338e-06]]]
[[[5.58435386e-06 5.66553672e-06 5.79407651e-06 ... 1.77805056e-03
   4.07589506e-03 1.57357205e-03]
  [4.46671748e-08 2.84792634e-08 1.60533684e-08 ... 5.45248855e-03
   7.92236347e-03 5.55569585e-03]
  [5.85560356e-09 1.24977424e-08 8.66914807e-09 ... 5.97133581e-03
   5.13985427e-03 7.11186696e-03]
  ...
  [3.46402657e-10 5.95331451e-10 6.47839227e-10 ... 8.80935622e-05
   5.80802

In [ ]:
train_speakers_df_reset = train_speakers_df.reset_index()
test_speakers_df_reset = test_speakers_df.reset_index()

In [ ]:
cols_train_trimmed = [ pd.Series( mel_audio_trimmed_train.tolist() ) , pd.Series(native_language_codes_train) ]
train_speakers_df_trimmed = pd.concat( cols_train_trimmed, axis = 1 )
train_speakers_df_trimmed.columns = ["melspectogram", "native_language"]

cols_test_trimmed = [  pd.Series( mel_audio_trimmed_test.tolist() ) , pd.Series( native_language_codes_test ) ]
test_speakers_df_trimmed = pd.concat( cols_test_trimmed, axis = 1 )
test_speakers_df_trimmed.columns = ["melspectogram", "native_language"]

print( train_speakers_df_trimmed.head() )
print( test_speakers_df_trimmed.head() )

                                       melspectogram  native_language
0  [[[0.023143284022808075, 0.023346584290266037,...              0.0
1  [[[0.023143284022808075, 0.023346584290266037,...              0.0
2  [[[0.023143284022808075, 0.023346584290266037,...              0.0
3  [[[0.023143284022808075, 0.023346584290266037,...              0.0
4  [[[0.023143284022808075, 0.023346584290266037,...              1.0
                                       melspectogram  native_language
0  [[[5.584353857557289e-06, 5.6655367188795935e-...              0.0
1  [[[5.584353857557289e-06, 5.6655367188795935e-...              3.0
2  [[[5.584353857557289e-06, 5.6655367188795935e-...              4.0
3  [[[5.584353857557289e-06, 5.6655367188795935e-...              5.0
4  [[[5.584353857557289e-06, 5.6655367188795935e-...              6.0


In [ ]:
print( train_speakers_df_trimmed.head() )
print( test_speakers_df_trimmed.head() )

                                       melspectogram  native_language
0  [[[0.023143284022808075, 0.023346584290266037,...              0.0
1  [[[0.023143284022808075, 0.023346584290266037,...              0.0
2  [[[0.023143284022808075, 0.023346584290266037,...              0.0
3  [[[0.023143284022808075, 0.023346584290266037,...              0.0
4  [[[0.023143284022808075, 0.023346584290266037,...              1.0
                                       melspectogram  native_language
0  [[[5.584353857557289e-06, 5.6655367188795935e-...              0.0
1  [[[5.584353857557289e-06, 5.6655367188795935e-...              3.0
2  [[[5.584353857557289e-06, 5.6655367188795935e-...              4.0
3  [[[5.584353857557289e-06, 5.6655367188795935e-...              5.0
4  [[[5.584353857557289e-06, 5.6655367188795935e-...              6.0


In [ ]:
train_speakers_df_reset_trim_no_index = train_speakers_df_trimmed[[ "melspectogram", "native_language" ]]
test_speakers_df_reset_trim_no_index = test_speakers_df_trimmed[[ "melspectogram", "native_language" ]]

In [ ]:
print(  len( train_speakers_df_reset_trim_no_index.iloc[0, 0][0] ))
print( len(train_speakers_df_reset_trim_no_index.iloc[1, 0][0]) )
print( len(train_speakers_df_reset_trim_no_index.iloc[2, 0][0]) )
print(  len( train_speakers_df_reset_trim_no_index.iloc[0, 0][0][0] ))
print( len(train_speakers_df_reset_trim_no_index.iloc[1, 0][0][0]) )
print( len(train_speakers_df_reset_trim_no_index.iloc[2, 0][0][0]) )
print( train_speakers_df_reset_trim_no_index.iloc[0, 0] )
print( train_speakers_df_reset_trim_no_index.iloc[1, 0] )

128
128
128
185
185
185
[[[0.023143284022808075, 0.023346584290266037, 0.020850325003266335, 0.013113382272422314, 0.014230255037546158, 0.01911274343729019, 0.015044324100017548, 0.015299160964787006, 0.0282547939568758, 0.030713865533471107, 0.01242560800164938, 0.005610258784145117, 0.00851559080183506, 0.012552175670862198, 0.012957005761563778, 0.017562096938490868, 0.015147197060286999, 0.024925539270043373, 0.04742235690355301, 0.028474105522036552, 0.019907323643565178, 0.009117956273257732, 0.006554635241627693, 0.033490560948848724, 0.032042600214481354, 0.009151199832558632, 0.009496084414422512, 0.05248098820447922, 2.576683759689331, 5.418237209320068, 2.6949329376220703, 0.28203344345092773, 0.012769571505486965, 0.002404638100415468, 0.014233093708753586, 0.016203289851546288, 0.006594196427613497, 0.0040792315267026424, 0.025141170248389244, 0.0222383514046669, 0.01575687900185585, 0.01325801108032465, 0.005316688679158688, 0.007817068137228489, 0.01204034686088562, 0.0

In [ ]:
# need to convert dataframe to Dataset class
class AudioDataset(Dataset):
    """Audio dataset."""

    def __init__(self, dataframe):
        self.data_frame = dataframe

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        audio_data = self.data_frame.iloc[idx, 0]
        y_label = np.array( self.data_frame.iloc[idx, 1], dtype=np.double )
        sample = np.array(audio_data, dtype=np.double )

        return sample , y_label

In [ ]:
batch_size_train = 10
batch_size_test = 10
train_data = AudioDataset( train_speakers_df_reset_trim_no_index )
test_data = AudioDataset( test_speakers_df_reset_trim_no_index )
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size_test, shuffle=True)

In [ ]:
print( train_speakers_df['native_language'].unique()  )
print( len( train_speakers_df['native_language'].unique() ) )

['afrikaans' 'agni' 'akan' 'albanian' 'amazigh' 'amharic' 'arabic'
 'armenian' 'ashanti' 'azerbaijani' 'bafang' 'baga' 'bai' 'bambara'
 'bamun' 'bari' 'basque' 'bavarian' 'belarusan' 'bengali' 'bosnian'
 'bulgarian' 'burmese' 'cantonese' 'carolinian' 'catalan' 'cebuano'
 'chaldean' 'chamorro' 'chichewa' 'chittagonian' 'croatian' 'czech'
 'danish' 'dari' 'dinka' 'dutch' 'ebira' 'edo' 'english']
40


In [ ]:
print( train_speakers_df_reset_trim_no_index['native_language'].unique()  )
print( len( train_speakers_df_reset_trim_no_index['native_language'].unique() ) )

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39.]
40


In [ ]:
print( len( speakers_valid_df_subset['melspectogram'].iloc[0] ) )

128


In [ ]:
import torch.nn.functional as F
from torch.nn import init

# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization
        #self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        #self.conv1 = nn.Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)) # 1 channel
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        # Convolution Block 2
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        # Convolution Block 3
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Convolution Block 4
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        #self.lin = nn.Linear(in_features=64, out_features=10)
        self.lin = nn.Linear(in_features=64, out_features=40)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = x.view(x.shape[0], -1)

        # Linear layer
        x = self.lin(x)

        # Final output
        return x

# Create the model and put it on the GPU if available
myModel = AudioClassifier()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
# Check that it is on Cuda
next(myModel.parameters()).device

device(type='cpu')

In [ ]:
def training(model, train_dl, num_epochs):
  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  # Repeat for each epoch
  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0

    # Repeat for each batch in the training set
    for i, (inputs_in, labels_in) in enumerate(train_dl):
        # Get the input features and target labels, and put them on the GPU

        # already tensors below so no need
        #inputs = inputs_in.to(device)
        #labels = torch.from_numpy( np.array(  labels_in ) ).to(device) # i, data in enumerate(train_dl)

        inputs = inputs_in.float()
        labels = labels_in.long()

        # Normalize the inputs
        #inputs_m, inputs_s = inputs.mean(), inputs.std()
        #inputs = (inputs - inputs_m) / inputs_s

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs )
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()

        # Get the predicted class with the highest score
        _, prediction = torch.max(outputs,1)
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

        #if i % 10 == 0:    # print every 10 mini-batches
        #    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
    
    # Print stats at the end of the epoch
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

  print('Finished Training')
  
num_epochs=10   # Just for demo, adjust this higher.
training(myModel, train_loader , num_epochs)

Epoch: 0, Loss: 3.71, Accuracy: 0.02
Epoch: 1, Loss: 3.64, Accuracy: 0.14
Epoch: 2, Loss: 3.30, Accuracy: 0.31
Epoch: 3, Loss: 2.68, Accuracy: 0.31
Epoch: 4, Loss: 2.43, Accuracy: 0.31
Epoch: 5, Loss: 2.39, Accuracy: 0.31
Epoch: 6, Loss: 2.39, Accuracy: 0.31
Epoch: 7, Loss: 2.40, Accuracy: 0.31
Epoch: 8, Loss: 2.40, Accuracy: 0.31
Epoch: 9, Loss: 2.40, Accuracy: 0.31
Finished Training


In [ ]:
#sources
#https://www.kaggle.com/rtatman/speech-accent-archive
#https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5